In [50]:
import pandas as pd
import numpy as np
import pprint
import itertools

In [60]:
class Seeder:
    def __init__(self):
        self.seeds = set()
        self.cache = dict()

    def get_seed(self, word):
        LIMIT = 2
        seed = self.cache.get(word,None)
        if seed is not None:
            return seed
        for seed in self.seeds:
            if self.distance(seed, word) <= LIMIT:
                self.cache[word] = seed
                return seed
        self.seeds.add(word)
        self.cache[word] = word
        return word

    def distance(self, s1, s2):
        l1 = len(s1)
        l2 = len(s2)
        matrix = [list(range(zz,zz + l1 + 1)) for zz in list(range(l2 + 1))]
        for zz in list(range(0,l2)):
            for sz in list(range(0,l1)):
                if s1[sz] == s2[zz]:
                    matrix[zz+1][sz+1] = min(matrix[zz+1][sz] + 1, matrix[zz][sz+1] + 1, matrix[zz][sz])
                else:
                    matrix[zz+1][sz+1] = min(matrix[zz+1][sz] + 1, matrix[zz][sz+1] + 1, matrix[zz][sz] + 1)
        return matrix[l2][l1]

In [18]:
main_dataset = pd.read_csv('principal.csv', encoding= 'utf-8')
main_dataset.head(3)

,crash_date_time,crash_date,crash_time,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,contributing_factor_vehicle_3,vehicle_type_code_3,cross_street_name,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5,day,no_location,day_time
0,2022-06-29 06:55:00,2022-06-29,6:55,THROGS NECK BRIDGE,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,0,Night
1,2022-06-30 00:17:00,2022-06-30,0:17,WYONA STREET,1.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,1,Night
2,2021-04-14 05:32:00,2021-04-14,5:32,BRONX WHITESTONE BRIDGE,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,0,Night


In [21]:
main_dataset.rename(columns= {'crash_date_time': 'colision_fecha_hora',
 'crash_date': 'colision_fecha',
 'crash_time': 'colision_hora',
 'on_street_name': 'nombre_de_calle',
 'number_of_persons_injured': 'numero_de_personas_heridas',
 'number_of_persons_killed': 'numero_de_personas_fallecidas',
 'number_of_pedestrians_injured': 'numero_de_transeuntes_heridos',
 'number_of_pedestrians_killed': 'numero_de_transeuntes_fallecidos',
 'number_of_cyclist_injured': 'numero_de_ciclistas_heridos',
 'number_of_cyclist_killed': 'numero_de_ciclistas_fallecidos',
 'number_of_motorist_injured': 'numero_de_conductores_heridos',
 'number_of_motorist_killed': 'numero_de_conductores_fallecidos',
 'contributing_factor_vehicle_1': 'factor_contribuyente_vehiculo_1',
 'contributing_factor_vehicle_2': 'factor_contribuyente_vehiculo_2',
 'collision_id': 'colision_ID',
 'vehicle_type_code1': 'codigo_tipo_de_vehiculo_1',
 'vehicle_type_code2': 'codigo_tipo_de_vehiculo_2',
 'borough': 'localidad',
 'zip_code': 'codigo_postal',
 'latitude': 'latitud',
 'longitude': 'longitud',
 'location': 'ubicacion',
 'contributing_factor_vehicle_3': 'factor_contribuyente_vehiculo_3',
 'vehicle_type_code_3': 'codigo_tipo_de_vehiculo_3',
 'contributing_factor_vehicle_4': 'factor_contribuyente_vehiculo_4',
 'vehicle_type_code_4': 'codigo_tipo_de_vehiculo_4',
 'contributing_factor_vehicle_5': 'factor_contribuyente_vehiculo_5',
 'vehicle_type_code_5': 'codigo_tipo_de_vehiculo_5',
 'day': 'dia',
 'no_location': 'sin_ubicacion',
 'day_time': 'hora_del_dia'}, inplace= True)
main_dataset.head(1)

,colision_fecha_hora,colision_fecha,colision_hora,nombre_de_calle,numero_de_personas_heridas,numero_de_personas_fallecidas,numero_de_transeuntes_heridos,numero_de_transeuntes_fallecidos,numero_de_ciclistas_heridos,numero_de_ciclistas_fallecidos,...,factor_contribuyente_vehiculo_3,codigo_tipo_de_vehiculo_3,cross_street_name,factor_contribuyente_vehiculo_4,codigo_tipo_de_vehiculo_4,factor_contribuyente_vehiculo_5,codigo_tipo_de_vehiculo_5,dia,sin_ubicacion,hora_del_dia
0,2022-06-29 06:55:00,2022-06-29,6:55,THROGS NECK BRIDGE,0.0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,0,Night


In [23]:
main_dataset.hora_del_dia.replace(['Night', 'Day'], ['Noche', 'Dia'], inplace= True)
main_dataset.hora_del_dia.unique()

array(['Noche', 'Dia'], dtype=object)

In [56]:
vehicles = list(main_dataset.codigo_tipo_de_vehiculo_1.unique()) + list(main_dataset.codigo_tipo_de_vehiculo_2.unique()) + list(main_dataset.codigo_tipo_de_vehiculo_3.unique()) + list(main_dataset.codigo_tipo_de_vehiculo_4.unique()) + list(main_dataset.codigo_tipo_de_vehiculo_5.unique())
firetruck = ['Fire', 'fire', 'FIRE']
vehicles = list(set(vehicles))
vehicles = [str(code) for code in vehicles]
len(vehicles)

1535

In [39]:
output = {}
def split(word):
    return [char for char in word]

for word in vehicles:
    ascending_word = split(word)
    unique = "".join(set(sorted(ascending_word)))
    if unique not in output:
        output[unique] = []
    output[unique].append(word)

print(list(output.values()))

[['EXCAVATOR'], ['Fire Depar'], ['motorcycle'], ['COMM', 'COM'], ['U-HAUL TRU'], ['Truck'], ['box t'], ['FOR VAN'], ['VS2'], ['E450'], ['trail'], ['E-SKATEBOA'], ['MTA WORK T'], ['FREIGHT'], ['4DS'], ['skateboard'], ['TRAILER/FL'], ['Horse trai'], ['USPCS'], ['Convertible'], ['FLATBED FR', 'FLATBED TR'], ['BOB C'], ['OMR'], ['UPS Van'], ['E-BIK', 'E-BIKE'], ['spc'], ['DUMP TRUCK'], ['fire'], ['yellow cab'], ['itas'], ['MARKED VAN'], ['PEDI CAB'], ['FIRE APPAR'], ['NJ transit'], ['rep'], ['comm', 'com'], ['Fedex'], ['UNKOW', 'UNKNOW', 'UNKOWN', 'UKNOWN', 'UNKNOWN'], ['POWER'], ['vespa'], ['MCR'], ['Livery Omn'], ['MINIVAN'], ['BUMP'], ['MOPET'], ['Bobcat'], ['DSNY sanit'], ['FED EX'], ['�MBU'], ['E SCOOTER', 'ESCOOTER S'], ['PKUP'], ['pedicab'], ['Taxi'], ['courier'], ['pas'], ['BUs'], ['GAS SCOOTE'], ['e-bike'], ['Garbage or Refuse'], ['REFG'], ['Self insur'], ['FD TR'], ['SCHOO'], ['Bike'], ['HINO TANK'], ['delviery', 'delivery'], ['Road sweep'], ['E-Scooter', 'E-Scoter'], ['Razor sco

In [58]:
import collections
groups = collections.defaultdict(set)
for word in vehicles:
    groups[frozenset(word)].add(word)

print(groups)

defaultdict(<class 'set'>, {frozenset({'E', 'R', 'V', 'X', 'O', 'T', 'C', 'A'}): {'EXCAVATOR'}, frozenset({' ', 'r', 'i', 'p', 'F', 'a', 'e', 'D'}): {'Fire Depar'}, frozenset({'c', 'r', 'o', 't', 'y', 'm', 'l', 'e'}): {'motorcycle'}, frozenset({'C', 'M', 'O'}): {'COMM', 'COM'}, frozenset({'H', ' ', 'U', 'R', 'L', '-', 'T', 'A'}): {'U-HAUL TRU'}, frozenset({'c', 'r', 'k', 'u', 'T'}): {'Truck'}, frozenset({' ', 'o', 't', 'b', 'x'}): {'box t'}, frozenset({' ', 'N', 'R', 'V', 'F', 'O', 'A'}): {'FOR VAN'}, frozenset({'S', 'V', '2'}): {'VS2'}, frozenset({'5', '4', 'E', '0'}): {'E450'}, frozenset({'r', 'i', 't', 'l', 'a'}): {'trail'}, frozenset({'B', 'E', 'S', '-', 'K', 'O', 'T', 'A'}): {'E-SKATEBOA'}, frozenset({'W', ' ', 'R', 'M', 'K', 'O', 'T', 'A'}): {'MTA WORK T'}, frozenset({'H', 'I', 'E', 'R', 'G', 'F', 'T'}): {'FREIGHT'}, frozenset({'S', '4', 'D'}): {'4DS'}, frozenset({'r', 'o', 'k', 's', 't', 'b', 'd', 'a', 'e'}): {'skateboard'}, frozenset({'I', 'E', 'R', 'F', 'L', '/', 'T', 'A'}): {

In [61]:
def group_similar(words):
    seeder = Seeder()
    words = sorted(words, key=seeder.get_seed)
    groups = itertools.groupby(words, key=seeder.get_seed)
    return [list(v) for k,v in groups]

print(pprint.pprint(group_similar(vehicles), width= 120))

[["''lime mope"],
 ['11 PA'],
 ['12 PASS VA'],
 ['12 PASSENG'],
 ['12 passage'],
 ['16M'],
 ['18 WEELER', '18 WHEELER'],
 ['18 wheel'],
 ['197209'],
 ['2 HOR'],
 ['2 dr sedan', '4 dr sedan'],
 ['2015'],
 ['2DCP'],
 ['3 WHE'],
 ['3 Wheel Sc'],
 ['3-Door'],
 ['325FLCR'],
 ['35 FT'],
 ['38AB-'],
 ['4DS', 'RDS', 'D', 'D1', 'HD', 'DL', 'RD/S', 'EMS', 'UPS', 'PAS'],
 ['4dsd', 'psd'],
 ['50 CC Moto'],
 ['50CC SCOOT'],
 ['50cc Scoot'],
 ['52? t'],
 ['600AJ'],
 ['72000', '00000'],
 ['7532433'],
 ['997', '7', '999', '994'],
 ['ACCES'],
 ['ACCESS A R'],
 ['AERIAL LIF'],
 ['ALUMI'],
 ['AMAZON SPR'],
 ['AMBULCANCE',
  'AMBULAMCE',
  'AMBULACE',
  'ABULANCE',
  'AMBULANCE`',
  'AMBULENCE',
  'EMBULANCE',
  'AMBULANCE',
  'AMBUKANCE',
  'AMUBULANCE',
  'AMBULANE'],
 ['AMBULETTE'],
 ['AMbulance'],
 ['APPOR', 'ARMOR', 'APP'],
 ['APURP'],
 ['ART M'],
 ['ASPHA'],
 ['ASPHALT RO'],
 ['ASTRO'],
 ['AUTOTRLR T'],
 ['Amazon tru'],
 ['Ambiance', 'Ambulance'],
 ['Ambu', 'ambul', 'ambu', 'amb', 'Amb', 'Ambul'],
 

In [7]:
lista_calles = list(main_dataset.nombre_calle.unique())
print(len(lista_calles))
print(len(list(main_dataset.nombre_calle.unique())))

11489
11489


In [16]:
main_dataset_locacion = main_dataset[['nombre_calle', 'latitud', 'longitud']]
main_dataset_locacion.head(3)

,nombre_calle,latitud,longitud
0,THROGS NECK BRIDGE,NaN,NaN
1,WYONA STREET,40.669792,-73.8924
2,BRONX WHITESTONE BRIDGE,NaN,NaN


In [17]:
main_dataset_locacion.to_csv('main_dataset_locacion.csv', encoding= 'utf-8', index= False)

In [20]:
list(main_dataset.off_street_name.unique())

[nan,
 'SUTTER AVENUE',
 'Sagamore avenue',
 'UNIVERSITY AVENUE',
 'ANTHONY STREET',
 '31ave',
 'GRAND CENTRAL PARKWAY',
 'EAST 21 STREET',
 'OSGOOD AVENUE',
 'CONCORD STREET',
 '\x7f�ST 138 STREET',
 'JORALEMON STREET',
 'AVENUE K',
 'HYLAN BOULEVARD',
 'LONGWOOD AVENUE',
 'ROPES AVENUE',
 'EAST 108 STREET',
 'BROADWAY',
 'WOOSTER STREET',
 '58 STREET',
 'CHURCH AVENUE',
 'PELHAM PARKWAY NORTH',
 'BLAKE AVENUE',
 'BUFFALO AVENUE',
 'WHITE PLAINS ROAD',
 'SOUTH 5 STREET',
 '184 STREET',
 '155 STREET',
 'STUYVESANT AVENUE',
 '24 AVENUE',
 'PARK AVENUE',
 'SYCAMORE STREET',
 'STEWART AVENUE',
 '85 STREET',
 'GROVE STREET',
 'ANDREWS AVENUE',
 '64 PLACE',
 'BEVERLEY ROAD',
 '11 AVENUE',
 'EAST 36 STREET',
 '74 STREET',
 'MADISON AVENUE',
 'SHEEPSHEAD BAY ROAD',
 'BACHE AVENUE',
 'AVENUE L',
 'JAY STREET',
 'JOHNSON AVENUE',
 'VIRGINIA AVENUE',
 'EAST 8 STREET',
 '65 STREET',
 'CONWAY STREET',
 'WEST 161 STREET',
 'PLATINUM AVENUE',
 'MOUNT MORRIS PARK WEST',
 '1 AVENUE',
 'HILLCREST STREE